# Run logistic regression on fraction labels

Run the logistic regression as the first model for the project. It will be used to compare with the deep learning model.

## Import necessary modules

In [1]:
# python built-in libraries
import datetime

In [2]:
# choose the tensorflow log level
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

In [3]:
# import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# ohter packages
import numpy as np

import tfhelper.core as tfhelper

## Global Variables

In [5]:
# sampling
FEATURES_FILE = "./data/tensors/features.npy"  # path to the file of the feature tensor
LABELS_FILE = "./data/tensors/labels.npy"  # path to the file of the feature tensor
FRACTIONS = (0.8, 0.1, 0.1)  # train, validation, test
BATCH_SIZE = 128  # size of the batch
BUFFER_SIZE = BATCH_SIZE * 2  # size of the shuffle buffer
# training
LEARNING_RATE = 0.001  # starting learning rate
BETA1 = 0.9  # decay 1
BETA2 = 0.999  # decay 2
EPOCHS = 200  # number of epochs
# saving
TIME_STAMP = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  # time stamp
SAVE_LOC = "./data/models/lr_{}".format(TIME_STAMP)  # path to the folder to save the model
LOG_LOC = "./data/logs/fit/lr_{}".format(TIME_STAMP)  # path to the log, if you change this, you also need to change it in the run_tensorboard

## Obtain data

The data is a 209 data point PDF. The label is a one-hot 2 dim vector. `10` means major phase >= threshold, `01` means major phase <= threshold.

In [6]:
def load_data():
    # load
    features, labels = np.load(FEATURES_FILE), np.load(LABELS_FILE)
    # shuffle
    n = features.shape[0]
    shuffled_idx = np.random.permutation(n)
    features, labels = features[shuffled_idx], labels[shuffled_idx]
    # split
    f0, f1, f2 = FRACTIONS
    i, j, k = round(f0 * n), round((f0 + f1) * n), round((f0 + f1 + f2) * n)
    train_data = tf.data.Dataset.from_tensor_slices((features[:i], labels[:i])).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    valid_data = tf.data.Dataset.from_tensor_slices((features[i:j], labels[i:j])).batch(BATCH_SIZE)
    test_data = tf.data.Dataset.from_tensor_slices((features[j:k], labels[j:k])).batch(BATCH_SIZE)
    return train_data, valid_data, test_data

In [7]:
train_data, valid_data, test_data = load_data()

## Create the model

We use the logistric regression. It is a single layer with a softmax function.

In [8]:
def create_model():
    model = keras.Sequential()
    model.add(keras.Input(shape=(209,)))
    model.add(layers.Dense(2, activation="softmax"))
    return model

In [9]:
model = create_model()

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 420       
Total params: 420
Trainable params: 420
Non-trainable params: 0
_________________________________________________________________


## Choose optimization method

In [11]:
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(LEARNING_RATE, BETA1, BETA2),
    # Loss function to minimize
    loss=keras.losses.CategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[
        keras.metrics.CosineSimilarity()
    ],
)

## Train the model

In [12]:
# tensor board
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_LOC, histogram_freq=1)

In [13]:
# early stopping to avoid over fitting
earlystopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

In [14]:
history = model.fit(
    x=train_data,
    epochs=EPOCHS,
    callbacks=[
        tensorboard_callback,
        earlystopping_callback
    ],
    validation_data=valid_data
)

Epoch 1/200
688/688 [==============================] - 13s 13ms/step - loss: 0.6606 - cosine_similarity: 0.7303 - val_loss: 0.6506 - val_cosine_similarity: 0.7359
Epoch 2/200
688/688 [==============================] - 3s 4ms/step - loss: 0.6481 - cosine_similarity: 0.7375 - val_loss: 0.6476 - val_cosine_similarity: 0.7378
Epoch 3/200
688/688 [==============================] - 3s 4ms/step - loss: 0.6458 - cosine_similarity: 0.7389 - val_loss: 0.6470 - val_cosine_similarity: 0.7382
Epoch 4/200
688/688 [==============================] - 3s 4ms/step - loss: 0.6446 - cosine_similarity: 0.7396 - val_loss: 0.6473 - val_cosine_similarity: 0.7380
Epoch 5/200
688/688 [==============================] - 3s 4ms/step - loss: 0.6437 - cosine_similarity: 0.7402 - val_loss: 0.6467 - val_cosine_similarity: 0.7384
Epoch 6/200
688/688 [==============================] - 3s 4ms/step - loss: 0.6432 - cosine_similarity: 0.7404 - val_loss: 0.6465 - val_cosine_similarity: 0.7385
Epoch 7/200
688/688 [===========

## Evaluate the model

In [15]:
result = model.evaluate(
    x=test_data,
    return_dict=True
)

86/86 [==============================] - 0s 3ms/step - loss: 0.6432 - cosine_similarity: 0.7406


In [16]:
for name, val in result.items():
    print("{:20s} {:.4f}".format(name, val))

loss                 0.6432
cosine_similarity    0.7406


## Save the model

In [17]:
model.save(SAVE_LOC)

INFO:tensorflow:Assets written to: ./data/models/lr_20210914-171635/assets
